# Linear Solves in UnifyML

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Linear_Solves.ipynb)
&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)

Linear solves are a vital part in many simulations and machine learning applications.
UnifyML provides an easy-to-use interface for performing linear solves that supports backpropagation via implicit gradients.
Dense, sparse, and matrix-free linear systems can be solved this way.


In [1]:
%%capture
!pip install unifyml

In [2]:
from unifyml import math
from unifyml.math import wrap, channel, dual, Solve, const_vec, tensor

Linear solves and sparse matrices are supported on all backends.
Feel free to choose the below line to use `jax`, `tensorflow` or `numpy` instead.

In [3]:
math.use('torch')

torch

We can perform a linear solve by passing a matrix `A`, right-hand-side vector `b` and initial guess `x0` to [`solve_linear()`](unifyml/math/#unifyml.math.solve_linear).

We recommend passing UnifyML tensors. Then, the [dual dimensions of the matrix](Matrices.html#Primal-and-Dual-Dimensions) must match the initial guess and the primal dimensions must match the right-hand-side.

Alternatively, `solve_linear()` can be used called with native tensors (see [below](#Linear-Solves-with-Native-Tensors)).

In [4]:
A = tensor([[0, 1], [1, 0]], channel('b_vec'), dual('x_vec'))
b = tensor([2, 3], channel('b_vec'))
x0 = tensor([0, 0], channel('x_vec'))
math.solve_linear(A, b, Solve(x0=x0))

(3.000, 2.000) along b_vecᶜ

UnifyML implements multiple algorithms to solve linear systems, such as the conjugate gradient method (`CG`) and the stabilized bi-conjugate gradient method (`biCG`).
All SciPy solvers are also available.
For a full list, see [here](unifyml/math/#unifyml.math.solve_linear).

In [5]:
math.solve_linear(A, b, Solve('CG', x0=x0))

(3.000, 2.000) along b_vecᶜ

In [6]:
math.solve_linear(A, b, Solve('biCG-stab', x0=x0))

(3.000, 2.000) along b_vecᶜ

In [7]:
math.solve_linear(A, b, Solve('scipy-GMres', x0=x0))

(3.000, 2.000) along b_vecᶜ

## Matrix-free Solves

Instead of passing a matrix, you can also specify a linear Python function that computes the matrix-vector product.
This will typically be slower unless the function is compiled to a matrix.

In [8]:
def linear_function(x):
    return x * (2, 1)

math.solve_linear(linear_function, b, Solve(x0=x0))

(1.000, 3.000) along b_vecᶜ

## Explicit Matrices from Python Functions

UnifyML can also [build an explicit matrix representation](Matrices.html#Building-Matrices-from-Linear-Functions) of the provided Python function.
You can do this either by explicitly obtaining the matrix first using [`matrix_from_function`](unifyml/math#unifyml.math.matrix_from_function) or by annotating the linear function with
[`jit_compile_linear`](unifyml/math#unifyml.math.jit_compile_linear).
If the function adds a constant offset to the output, this will automatically be subtracted from the right-hand-side vector.

In [9]:
from unifyml.math import jit_compile_linear

math.solve_linear(jit_compile_linear(linear_function), b, Solve(x0=x0))

C:\PhD\UnifyML\unifyml\backend\torch\_torch_backend.py:752: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:56.)
  return torch.sparse_csr_tensor(row_pointers, column_indices, values, shape, device=values.device)


(1.000, 2.000, 1.500, 3.000) (b_vecᶜ=2, x_vecᶜ=2)

## Preconditioned Linear Solves

UnifyML includes an ILU and experimental cluster preconditioner.
To use a preconditioner, simply specify `preconditioner='ilu'` when creating the `Solve` object.

In [10]:
math.solve_linear(jit_compile_linear(linear_function), b, Solve('scipy-CG', x0=x0, preconditioner='ilu'))

C:\ProgramData\Anaconda3\envs\phiflow2\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:590: SparseEfficiencyWarning: CSR matrix format is required. Converting to CSR matrix.
  warn('CSR matrix format is required. Converting to CSR matrix.',


(1.000, 2.000, 1.500, 3.000) (b_vecᶜ=2, x_vecᶜ=2)

The ILU preconditioner always runs on the CPU and should be paired with a SciPy linear solver for optimal efficiency.
Available SciPy solvers include `'scipy-direct'`, `'scipy-CG'`, `'scipy-GMres'`, `'scipy-biCG'`, `'scipy-biCG-stab'`, `'scipy-CGS'`, `'scipy-QMR'`, `'scipy-GCrotMK'` (see the [API](unifyml/math/index.html#unifyml.math.solve_linear)).

If the matrix or linear function is constant, i.e. only [depends on NumPy arrays](NumPy_Constants.html), the preconditioner computation can be performed during [JIT compilation](JIT.html).

[](Matrices.html#Building-Matrices-from-Linear-Functions))

In [11]:
@math.jit_compile
def jit_perform_solve(b):
    return math.solve_linear(jit_compile_linear(linear_function), b, Solve('scipy-CG', x0=x0, preconditioner='ilu'))

jit_perform_solve(b)

C:\PhD\UnifyML\unifyml\backend\torch\_torch_backend.py:60: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  tensor = torch.from_numpy(x)
C:\PhD\UnifyML\unifyml\backend\torch\_torch_backend.py:752: TracerWarning: torch.sparse_csr_tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.sparse_csr_tensor(row_pointers, column_indices, values, shape, device=values.device)
C:\PhD\UnifyML\unifyml\backend\torch\_torch_backend.py:125: TracerWarning: Converting a tensor to a NumPy array might cause the t

(1.000, 2.000, 1.500, 3.000) (b_vecᶜ=2, x_vecᶜ=2)

Here, the ILU preconditioner is computed during JIT-compile time since the linear function does not depend on `b`.

## Implicit Differentiation

UnifyML enables backpropagation through linear solves.
Instead of backpropagating through the unrolled loop (which can lead to inaccurate results and cause high memory consumption), Unify runs an ajoint linear solve for the pullback operation.

In [12]:
def loss_function(b):
    x = math.solve_linear(jit_compile_linear(linear_function), b, Solve(x0=x0))
    return math.l2_loss(x)

gradient_function = math.gradient(loss_function, 'b', get_output=False)
gradient_function(b)

(2.500, 3.750) along b_vecᶜ

## Matrix Gradients

UnifyML can also compute gradients for the (sparse) matrix used in a linear solve, which allows differentiating w.r.t. parameters that influenced the matrix values via backpropagation.
To enable this, pass `grad_for_f=True` to the `solve_linear()` call.


In [13]:
@math.jit_compile_linear
def conditioned_linear_function(x, conditioning):
    return x * conditioning

def loss_function(conditioning):
    b = math.ones_like(conditioning)
    x = math.solve_linear(conditioned_linear_function, b, Solve(x0=x0), conditioning=conditioning, grad_for_f=True)
    return math.l2_loss(x)

gradient_function = math.gradient(loss_function, 'conditioning', get_output=False)
gradient_function(tensor([1., 2.], channel('x_vec')))

(1.000, 0.125) along x_vecᶜ

## Obtaining Additional Information about a Solve

All solves are logged internally and can be shown by setting [`unifyml.set_logging_level('debug')`](unifyml#unifyml.set_logging_level).

Additional solve properties can be recorded using a [`SolveTape`](unifyml/math#unifyml.math.SolveTape).
Recording the full optimization trajectory requires setting `record_trajectories=True`.

In [17]:
import unifyml
unifyml.set_logging_level('debug')

with math.SolveTape() as solves:
    math.solve_linear(jit_compile_linear(linear_function), b, Solve('scipy-CG', x0=0 * b, preconditioner='ilu'))

factor_ilu: auto-selecting iterations=1 (eager mode) for matrix (2.000, 0.000); (0.000, 1.000) (vectorᶜ=2, ~vectorᵈ=2) (DEBUG), 2023-07-09 17:41:40,829n

Running forward pass of custom op forward '_matrix_solve_forward' given args ('y',) containing 1 native tensors (DEBUG), 2023-07-09 17:41:40,839n

Performing linear solve scipy-CG with tolerance float64 1e-05 (rel), float64 1e-05 (abs), max_iterations=1000 with backend torch (DEBUG), 2023-07-09 17:41:40,844n



C:\ProgramData\Anaconda3\envs\phiflow2\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:590: SparseEfficiencyWarning: CSR matrix format is required. Converting to CSR matrix.
  warn('CSR matrix format is required. Converting to CSR matrix.',


The solve information about a performed solve(s) can then be obtained by indexing specific solves by index or `Solve` object.

In [18]:
print(solves[0].solve)
print("Solution", solves[0].x)
print("Residual", solves[0].residual)
print("Fun.evals", solves[0].function_evaluations)
print("Iterations", solves[0].iterations)
print("Diverged", solves[0].diverged)
print("Converged", solves[0].converged)

scipy-CG with tolerance float64 1e-05 (rel), float64 1e-05 (abs), max_iterations=1000
Solution (1.000, 3.000)
Residual (0.000, 0.000)
Fun.evals 2
Iterations 1
Diverged False
Converged True


## Linear Solves with Native Tensors

When performing a linear solve without UnifyML tensors, the matrix must have shape (..., N, N) and `x0` and `b` must have shape `(..., N)` where `...` denotes the batch dimensions.
This matches the signatures of the native solve functions like `torch.linalg.solve` or `jax.numpy.linalg.solve`.

In [16]:
import torch

A = torch.tensor([[0., 1], [1, 0]])
b = torch.tensor([2., 3])
x0 = torch.tensor([0., 0])
math.solve_linear(A, b, Solve(x0=x0))

Running forward pass of custom op forward '_matrix_solve_forward' given args ('y', 'matrix') containing 2 native tensors (DEBUG), 2023-07-09 17:41:08,054n

Performing linear solve auto with tolerance float64 1e-05 (rel), float64 1e-05 (abs), max_iterations=1000 with backend torch (DEBUG), 2023-07-09 17:41:08,058n



tensor([3., 2.], device='cuda:0')

## Further Reading

We will upload a whitepaper to the ArXiv shortly, describing the implemented algorithms.

&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)